En la carpeta textos_para_analizar hay cuatro documentos. Cada uno de ellos corresponde a unx autorx y contiene, en cada línea, el nombre de un cuento. La idea es pasar cada cuento a un vector para luego ver cómo se agrupan.

Pasos a seguir:

0.   Instalar e importar bibliotecas necesarias.

1.   Montar el drive, cargar el modelo de fasttext y abrir la carpeta compartida con los textos.

2.   Definir una función que tome como parámetro un archivo .txt y devuelva un diccionario {autorx: lista de todos los nombres de los cuentos que ahí aparecen}.

3.   Definir una función que tome el diccionario anterior y modifique la lista de manera que cada elemento quede "limpio" y pueda usarse en `vector_cuento()`. Para eso, tener en cuenta mayúsculas, tildes, comas, guiones bajos, números, etcétera.

  Ejemplo: Si limpio el string 'Biografía de Tadeo Isidoro Cruz (1829-1874)' y lo meto en `vector_cuento()` debería internamente ir a https://ciudadseva.com/texto/biografia-de-tadeo-isidoro-cruz-1829-1874/


4.   En base a las funciones anteriores y a `vector_cuento()` (puede modificarse si hace falta), generar un diccionario {autorx: lista de embeddings de los textos de los cuentos correspondientes (un vector-embedding por texto)}. Guardar el diccionario en algún formato en la carpeta compartida. Este diccionario luego será levantado por una función que calculará las distancias semánticas entre vectores o directamente se puede graficar.

Para ejemplos de cuentos, entrar a https://ciudadseva.com/biblioteca/indice-autor-cuentos/

0) Instalar bs4 e importar las bibliotecas

In [ ]:
!pip install beautifulsoup4 #ejecutar esto una sola vez

import requests
from bs4 import BeautifulSoup
import time

!pip install fasttext
import fasttext

1) Montar drive. Cambiar los paths de la carpeta y del modelo según cómo los tengas en tu drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#paths
ft = fasttext.load_model('/content/drive/MyDrive/Colab/TallerNLP/cc.es.300.bin')
carpeta = '/content/drive/MyDrive/Colab/TallerNLP/cuentos_para_analizar/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2) Las siguientes funciones se encargan de buscar los archivos de los autores, leerlos y "limpiar" el nombre de los cuentos para poder ser usados en el item 3.

Resultado: diccionario de la forma {autorx, lista de sus cuentos}.

Importante: Requiere que dentro de la carpeta cuentos_para_analizar los unicos archivos .txt que existan sean el de los autores de la forma "autorx.txt" y dentro de cada archivo esten los nombres de los cuentos uno por cada linea.

In [ ]:
import os
import glob

def limpiar_texto(lista:list[str])->list[str]:
    """
    Se encarga de limpiar los nombres de los cuentos para que sean aptos para vector_cuentos()

    Entrada: lista_cuentos (list[str]), lista_cuentos = archivo.readlines() de los archivos con los cuentos de los autores

    Salida: lista con los nombres de los cuentos validos para vector_cuentos
    """
    char_a_reemplazar:dict[str,str] = {'ñ':'n','á':'a','é':'e','í':'i','ó':'o','ú':'u','\n':'',' ':'-','ã':'a','"':'','(':'',')':'','ä':'a','ë':'e','ï':'i','ö':'o','ü':'u',"'":""}
    res:list[str] = []
    for elemento in lista:
        aux:str = ""
        for letra in elemento:
            letra = letra.lower()
            if letra not in char_a_reemplazar.keys():
                aux += letra
            else:
                aux += char_a_reemplazar[letra]
        if len(aux)>0:
          res.append(aux.lower())
    return res

def obtener_autores(archivos_autores:list[str],carpeta:str)->list[str]:
    """
    Dada la lista de los archivos existentes, extrae el nombre del autor.
    Requiere que los unicos archivos dentro de la carpeta "cuentos_para_analizar" sean de la forma "autor.txt"
    y contenga los nombres de los cuentos, cada cuento en una linea del archivo

    Entrada: carpeta, donde estan los archivos (str).
    archivos_autores, con los archivos encontrados dentro de la carpeta (lista de str)

    Salida: lista con los nombres de los autores, extraidos de los archivos.
    """
    res:list[str] = []
    for dir_archivo in archivos_autores:
        autor:str = dir_archivo[len(carpeta):-4] #esto nose si es muy conveniente
        res.append(autor)
    return res

def cargar_nombres_cuentos(carpeta:str):
    """
    Dada una carpeta, busca los archivos de autores existentes guardando sus direcciones en una lista.
    Crea un diccionario de la forma {autor:lista_cuentos},
    obteniendo el nombre del autor usando obtener_autores y la lista_cuentos leyendo cada archivo y limpiando su texto con limpiar_cuentos

    Requiere que dentro de la carpeta los unicos archivos .txt existentes sean de la forma "autorx.txt"
    y su contenido sea linea por linea nombre de cuento, con cada linea con solo un cuento

    Entrada: carpeta (str).

    Salida: diccionario de la forma {autor:lista_cuentos_valida}, siendo lista_cuento_valida la lista con cuentos ya limpia para luego cada elemento ser utilizado en vector_cuento().
    """
    dicc:dict[str,list[str]] = {}
    archivos_txt:list[str] = glob.glob(os.path.join(carpeta, "*.txt")) #busca que archivos de texto hay en la carpeta
    autores:list[str] = obtener_autores(archivos_txt,carpeta)
    for i in range(0, len(archivos_txt)):
    #for i in range(0, 2):
        archivo = open(archivos_txt[i],"r", encoding="utf-8")
        dicc[autores[i]] = limpiar_texto(archivo.readlines())
        archivo.close()
    return dicc

dicc = cargar_nombres_cuentos(carpeta)
print(dicc)

{'cortazar': ['alguien-que-anda-por-ahi', 'almuerzos', 'amor-77', 'apocalipsis-de-solentiname', 'axolotl', 'bestiario', 'carta-a-una-senorita-en-paris', 'cartas-de-mama', 'casa-tomada', 'cefalea', 'circe', 'condor-y-cronopio', 'conducta-en-los-velorios', 'conservacion-de-los-recuerdos', 'continuidad-de-los-parques', 'cortisimo-metraje', 'datos-para-entender-a-los-perqueos', 'desayuno', 'deshoras', 'despues-del-almuerzo', 'educacion-de-principe', 'el-diario-a-diario', 'el-otro-cielo', 'el-perseguidor', 'en-un-vaso-de-agua-fria-o-preferentemente-tibia', 'epigrafe-de-rayuela', 'eugenesia', 'final-del-juego', 'grafitti', 'historia', 'historia-veridica', 'instrucciones-para-cantar', 'instrucciones-para-dar-cuerda-al-reloj', 'instrucciones-para-john-howell', 'instrucciones-para-llorar', 'instrucciones-para-subir-una-escalera', 'la-autopista-del-sur', 'la-caricia-mas-profunda', 'la-fe-en-el-tercer-mundo', 'la-isla-a-mediodia', 'la-noche-boca-arriba', 'la-puerta-condenada', 'la-salud-de-los-en

3) A partir de todas las funciones anteriores y la celda de más abajo queremos guardar un diccionario final {autorx:list[embedding_del_cuento]} en la carpeta compartida.

Comentario: Al final agregue lo que iria llamando a vector_cuento para cada libro y lo guardaría finalmente en un diccionario como se pide, pero no esta terminado!

In [50]:
def limpiar_html(texto) -> str:
    limpio = "\n\n".join(el.get_text(strip=True) for el in texto)
    return limpio

def vector_cuento(nombre_cuento,autor):
    """
    Con el nombre del cuento, entra a Ciudad Seva, extrae y limpia el texto del cuento.
    Luego pasa el texto a un vector y lo devuelve.

    Entrada: nombre de un cuento (str).

    Salida: embedding (vector) del cuento o None si hay una falla.
    """
    time.sleep(2)
    response = requests.get('https://ciudadseva.com/texto/' + nombre_cuento)
    if response.status_code != 200:
        print(f"Error en la conexión: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")

    # Locate the <article> with class including both "post-" and "texto"
    article = soup.find("article", class_=lambda x: x and "post-" in x and "texto" in x)
    if not article:
        print("Error. No se encontró el artículo")
        return None

    # Extract all relevant tags (e.g., paragraphs, headers, etc.)
    texto_completo = article.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6", "div", "blockquote"])
    # titulo = limpiar_html(texto_completo[0]).split('[')[0].strip()
    # autor = limpiar_html(texto_completo[0]).split(']')[-1].split('\n')[-1]
    #cuento = limpiar_html(texto_completo[3:-2].replace('\n',''))
    #print(limpiar_html(texto_completo[0]).split(']')[-1].split('\n')[-1])
    if autor !=limpiar_texto(limpiar_html(texto_completo[0]).split(']')[-1].split('\n')[-1].split(' '))[-1].lower():
      if nombre_cuento[len(nombre_cuento)-len(autor): len(nombre_cuento)] != autor:
        return vector_cuento(str(nombre_cuento+"-"+autor),autor)
      """else:
        print(str(nombre_cuento[0:len(nombre_cuento)-len(autor)-1]+"-2"),autor)
        return vector_cuento(str(nombre_cuento[0:len(nombre_cuento)-len(autor)-1]+"-2"),autor)"""

    #cuento = texto_completo[3:-2]
    cuento = limpiar_html(texto_completo[3:-2]).replace('\n', ' ')
    vector = ft[cuento]
    return vector

"""
def cargar_diccionario_con_embedings(dicc):
    res = {}
    for autor in dicc.keys():
        lista_embedings = []
        for cuento in dicc[autor]:
            lista_embedings.append(vector_cuento(cuento))
        res[autor] = lista_embedings
    return res


res = cargar_diccionario_con_embedings(dicc)"""

#no esta 100% terminado, tendria que ponerlo en una función
vectores_por_autor = {}
cuentos_no_encontrados = []
for autor, cuentos in dicc.items():
    vectores = []
    for cuento in cuentos:
        vector = vector_cuento(cuento,autor)
        if vector is not None:
            vectores.append(vector)
        else:
            print(f"❌ No se pudo obtener vector para '{cuento}' de {autor}")
            cuentos_no_encontrados.append((autor,cuento))
    vectores_por_autor[autor] = vectores

print(vectores_por_autor)
print(cuentos_no_encontrados)



Error en la conexión: 404
❌ No se pudo obtener vector para 'el-adivino' de borges
Error en la conexión: 404
❌ No se pudo obtener vector para 'silencio' de lispector
{'borges': [], 'lispector': []}
[('borges', 'el-adivino'), ('lispector', 'silencio')]
